In [ ]:
# %pip install selenium --q
# %pip install webdriver_manager --q

Note: you may need to restart the kernel to use updated packages.


In [10]:
from selenium import webdriver # para controlar o navegador (abrir, clicar, preencher, etc.).
from selenium.webdriver.common.by import By # usado para encontrar elementos na página (por ID, nome, classe, etc.).
from selenium.webdriver.support.ui import WebDriverWait # permite esperar até que algo apareça ou esteja pronto (evita usar sleep fixo).
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains #  usado para movimentos avançados como mouseover, drag-and-drop etc.
from selenium.webdriver.common.keys import Keys # para simular pressionamento de teclas (ex: ENTER).
from selenium.webdriver.support.ui import Select # para interagir com caixas de seleção (<select>).
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # iniciam o ChromeDriver automaticamente.
import time
import re
from bs4 import BeautifulSoup # para ler e analisar o HTML da página.
import json
import pandas as pd
import time
import os

In [11]:
# Definindo o caminho de download:

download_path = r"C:\Users\joao.beneton\Downloads\Chatbot"

In [14]:
# Configurando o Chrome e iniciando o navegador:

chrome_options = webdriver.ChromeOptions() # Criação do objeto, que permite configurar as opções do navegador.
prefs = {
    "download.default_directory": download_path, # Definição do diretório padrão onde os arquivos baixados serão salvos
    "download.prompt_for_download": False, # Desativação do prompt que aparece ao iniciar um download
    "profile.default_content_settings.popups": 0, # Desativação dos pop-ups relacionados ao download
    "safebrowsing.enabled": True, # Proteção contra sites maliciosos
}
chrome_options.add_experimental_option(
    "prefs", # O Chrome possui diversas "opções experimentais", e "prefs" é uma delas,
                # usada para definir preferências de comportamento do navegador.
        prefs
        ) # Adição das preferências

serviço = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(
    service=serviço,
    options=chrome_options
)

# Acesso a página do Banco Central:
# navegador = webdriver.Chrome()
# navegador.get('https://www.bcb.gov.br/estabilidadefinanceira/buscanormas?dataInicioBusca=14%2F02%2F1900&dataFimBusca=17%2F02%2F2025&tipoDocumento=Todos&refinadorRevogado=0')
# navegador.maximize_window()
# navegador.quit()


In [22]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    WebDriverException
)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


DOWNLOAD_DIR = os.path.abspath("Normas")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
    "plugins.plugins_disabled": ["Chrome PDF Viewer"]
}
options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(30)
driver.implicitly_wait(5)

averiguar = []

todos_os_links = ['https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Carta%20Circular&numero=3978']

for page_url in todos_os_links:
    try:
        driver.get(page_url)
    except (TimeoutException, WebDriverException) as e:
        print(f"[ERRO] não consegui carregar {page_url}: {e}")
        averiguar.append(page_url)
        continue

    pdfs_antes = set(f for f in os.listdir(DOWNLOAD_DIR) if f.lower().endswith(".pdf"))

    wait = WebDriverWait(driver, 20)
    try:
        btn = wait.until(
            EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Texto vigente compilado"))
        )
        btn.click()
        print(f"[OK] clique realizado em 'Texto vigente compilado' em {page_url}")
    except:
        try:
            btn = wait.until(
                EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Texto original"))
            )
            btn.click()
            print(f"[OK] clique realizado em 'Texto original' em {page_url}")
        except TimeoutException:
            print(f"[ERRO] botão de download não apareceu em {page_url}")
            averiguar.append(page_url)
            continue

    # 1. Espera o download INICIAR (o .crdownload aparecer)
    espera_inicio = 0
    download_iniciado = False
    while espera_inicio < 15:
        in_progress = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".crdownload")]
        if in_progress:
            print(f"[INFO] Download iniciado: {in_progress[0]}")
            download_iniciado = True
            break
        time.sleep(1)
        espera_inicio += 1

    # Se não começou em 15s, pode ser um download ultra-rápido
    if not download_iniciado:
        pdfs_depois_check = set(f for f in os.listdir(DOWNLOAD_DIR) if f.lower().endswith(".pdf"))
        novos_pdfs_check = pdfs_depois_check - pdfs_antes
        if novos_pdfs_check:
            print(f"[OK] Download ultra-rápido detectado: {novos_pdfs_check.pop()}")
            continue
        else:
            print(f"[ERRO] Download não iniciou em 15s para {page_url}")
            averiguar.append(page_url)
            continue

    # 2. Espera o download TERMINAR (o .crdownload desaparecer)
    espera_fim = 0
    while espera_fim < 60:
        in_progress = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".crdownload")]
        if not in_progress:
            break
        time.sleep(1)
        espera_fim += 1

    if espera_fim == 60:
        print(f"[ERRO] Download travou (ainda .crdownload) em {page_url}")
        averiguar.append(page_url)
        continue

    # 3. Verifica se um NOVO PDF foi realmente criado
    time.sleep(1)
    pdfs_depois = set(f for f in os.listdir(DOWNLOAD_DIR) if f.lower().endswith(".pdf"))
    novos_pdfs = pdfs_depois - pdfs_antes

    if not novos_pdfs:
        print(f"[ERRO] Download terminou, mas nenhum NOVO PDF foi encontrado em {page_url}")
        averiguar.append(page_url)
    else:
        print(f"[OK] baixado: {novos_pdfs.pop()}")
driver.quit()

print("\n===== LINKS PARA AVERIGUAR =====")
for url in averiguar:
    print(" -", url)

[OK] clique realizado em 'Texto original' em https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Carta%20Circular&numero=3978
[OK] Download ultra-rápido detectado: C_Circ_3978_v1_O.pdf

===== LINKS PARA AVERIGUAR =====


In [ ]:
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=4950',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=4955',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=4957',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=4958',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20CMN&numero=4995',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=69',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=82',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=84',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=198',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=199',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=200',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=202',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=229',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=239',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=266',
'Res. BCB 291, de 8 de fevereiro de 2023',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=307',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=313',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Resolu%C3%A7%C3%A3o%20BCB&numero=319',
'Res. BCB 436, de 28 de novembro de 2024',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Instru%C3%A7%C3%A3o%20Normativa%20BCB&numero=81',
'https://www.bcb.gov.br/estabilidadefinanceira/exibenormativo?tipo=Instru%C3%A7%C3%A3o%20Normativa%20BCB&numero=479',
''